# 融資融券明細

In [1]:
import time, os, pickle, glob
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np

import datetime
from datetime import timedelta

#! pip install chinesecalendar
import chinese_calendar
from chinese_calendar import is_workday

import warnings
warnings.filterwarnings('ignore')

In [8]:
%%time
chromedriver_autoinstaller.install()
chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(options=chrome_options)

Wall time: 1.87 s


## 抓表格 function

In [36]:
now = datetime.datetime.now().date()
def table(ticker, 
          start_year=now.year, start_month=now.month, start_day=now.day, 
          end_year=now.year, end_month=now.month, end_day=now.day):
    '''
    ticker: 股票號碼
    --------------
    (default 抓今天)
    start_year: 開始年份
    start_month: 開始月份
    start_day: 開始日期
    end_year: 結束年份
    end_month: 結束月份
    end_day: 結束日期
    '''
    
    url = f"https://stockchannelnew.sinotrade.com.tw/z/zc/zcn/zcn.djhtm?a={ticker}&c={start_year}-{start_month}-{start_day}&d={end_year}-{end_month}-{end_day}"
    browser.get(url)
    html = browser.page_source
    soup = bs(html, "lxml")

    try:
        # 值
        values = soup.find_all('td', "t3n1")
        value_list = [value.getText() for value in values]
        value_df = pd.DataFrame(np.array(value_list[:-2]).reshape(-1, 14))

        # 欄位名稱
        titles = soup.find_all('td', "t2")
        title_list = [title.getText() for title in titles][-14:] 
        title_all = ['融資' + title_list[i] if i+1 <= 7 else '融券' + title_list[i] for i in range(len(title_list)-1)] + ['資券' + title_list[-1]]

        # 日期
        dates = soup.find_all('td', "t3n0")[1:]  # 因為第一個是空格
        date_list = [date.getText().split('/') for date in dates]
        for i in range(len(date_list)):
            date_list[i][0] = str(int(date_list[i][0])+1911)
            date_list[i] = '-'.join(date_list[i])

        value_df.columns = title_all
        value_df.insert(0, "日期", date_list)  
        
        return value_df
    
    
    except Exception as e:
        #print('【系統訊息】{}'.format(e))
        return None

<hr>

## <font color = blue>第一次爬:

### 要爬的股票

In [28]:
path_from = os.getcwd() 
tickers = pd.read_csv(path_from + r'\台股全部4.csv', header=None)[0].tolist()

### 設定開始爬的股票號碼

In [29]:
start = 4142  # 自己輸入

start_index = int([i for (i, j) in enumerate(tickers) if j == start][0])
print(f'從第 {start_index+1} 個開始')

從第 5 個開始


### 事先設定

In [41]:
# 設定資料夾
path_go = f'{os.getcwd()}\\融資融券明細'   
if not os.path.exists(path_go) : 
    os.makedirs(path_go)

# 打開之前儲存查無資料的股票號碼繼續沿用
if not os.path.exists(r'3_problem_list.pkl') :
    problem = []
else:
    with open(r'3_problem_list.pkl', mode='rb') as inpf :  # 打開之前儲存的錯誤 dict 繼續沿用
        try:
            problem = pickle.load(inpf)
        except EOFError:
            problem = []

### 開始抓股票

In [42]:
for ticker in tickers[start_index:]:
    df = table(ticker, start_year=2022, start_month=1, start_day=1, end_year=2022, end_month=7, end_day=1) # 自訂!
    if isinstance(df, type(None)):
        problem.append(ticker)
        print(f'查無({ticker})融資融券資料')
        continue
    else:
        filename = str(ticker) + '-' + ''.join(df['日期'][len(df)-1].split('-')) + '_' + ''.join(df['日期'][0].split('-')) +'.csv'
        df.to_csv(f'{path_go}\\{filename}', encoding='cp950', index = False)
        print(f'成功匯出 {filename}')
    
    time.sleep(0.5)

成功匯出 4142-20220104_20220701.csv
成功匯出 4147-20220104_20220701.csv
成功匯出 4148-20220104_20220701.csv
查無(4150)融資融券資料
查無(4151)融資融券資料
成功匯出 4153-20220104_20220701.csv
查無(4154)融資融券資料
成功匯出 4155-20220104_20220701.csv
查無(4156)融資融券資料
成功匯出 4157-20220104_20220701.csv
查無(4160)融資融券資料
成功匯出 4161-20220104_20220701.csv
成功匯出 4162-20220104_20220701.csv
成功匯出 4163-20220104_20220701.csv
成功匯出 4164-20220104_20220701.csv
查無(4166)融資融券資料
成功匯出 4167-20220104_20220701.csv
成功匯出 4168-20220104_20220701.csv
查無(4169)融資融券資料
查無(4170)融資融券資料


KeyboardInterrupt: 

In [63]:
print(f'查無資料的股票號碼: {problem}')
with open(r'3_problem_list.pkl', mode='wb') as outf :
            pickle.dump(problem, outf)
print('')        
print(f'已將查無資料的股票號碼匯出為 3_problem_list.pkl')

查無資料的股票號碼: [4150, 4151, 4154, 4156, 4160, 4166, 4169, 4170]

已將查無資料的股票號碼匯出為 3_problem_list.pkl


<hr>

## <font color = blue>從現有的檔案更新 (自動更新到今天):

In [56]:
# 所有檔案
fnames = glob.glob(path_go + r'\*.csv', recursive=False)

# 目前共抓了哪些股票
tickers = [fname.split('\\')[-1].split('-')[0] for fname in fnames]

# 找出之前抓到哪一天
existfile_end = [fname.split('\\')[-1].split('_')[-1].split('.')[0] for fname in fnames]

# 新檔案該從哪一天開始抓
newfile_start = [datetime.datetime.strptime(i, "%Y%m%d") + datetime.timedelta(days=1) for i in existfile_end]

In [58]:
for i in list(zip(range(len(tickers)), tickers, newfile_start)):
    
    # i[0]: 第幾個位置、 i[1]:股票號碼  i[2]: 開始日期
    
    # 讀入舊的檔案
    exist_df = pd.read_csv(fnames[i[0]], sep=',', encoding='cp950', engine='python')
    # 新增的檔案
    add_df = table(i[1], start_year=i[2].year, start_month=i[2].month, start_day=i[2].day)
    # 合併檔案
    new_df = pd.concat([add_df, exist_df], axis=0, verify_integrity=True, ignore_index=True)
    # 新的檔案名
    filename = str(i[1]) + '-' + ''.join(new_df['日期'][len(new_df)-1].split('-')) + '_' + ''.join(new_df['日期'][0].split('-')) +'.csv'
    # 匯出檔案
    new_df.to_csv(f'{path_go}\\{filename}', encoding='cp950', index = False)
    # 移除舊的檔案
    os.remove(fnames[i[0]])
    
    print(f'{i[1]} 更新完成')
    time.sleep(0.5)

4153 更新完成
4155 更新完成
4157 更新完成
4161 更新完成
4162 更新完成
4163 更新完成
4164 更新完成
4167 更新完成
4168 更新完成
4171 更新完成
8027 更新完成
